In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import jax
import jax.numpy as jnp
import optax
import flax
from flax.training import train_state, checkpoints
from flax_rbf.flax_rbf import *
from irbfn_mpc.model import WCRBFNet
# from irbfn_mpc.irbfn_planner import IRBFNPlanner
import irbfn_mpc.irbfn_planner as ip

2024-09-05 20:09:51.042695: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
import gymnasium as gym
from irbfn_mpc.planner_utils import intersect_point, nearest_point
import numpy as np
import yaml
import argparse

In [4]:
from IPython.display import HTML, display
%matplotlib inline

In [5]:
env = gym.make(
    "f1tenth_gym:f1tenth-v0",
    config={
        "map": "Spielberg_blank",
        "observation_config": {"type": "dynamic_state"},
        "num_agents": 1,
        "control_input": ["accl", "steering_angle"],
    },
    render_mode="rgb_array",
)
env = gym.wrappers.RecordVideo(env, "video_irbfn_nmpc_newnewdata")

/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/irbfn/scripts/video_irbfn_nmpc_newnewdata folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [6]:
track = env.unwrapped.track
waypoints = np.stack([track.centerline.xs, track.centerline.ys, track.centerline.vxs, track.centerline.yaws]).T

In [7]:
# config_f = "configs/dnmpc_1regions_newdata_oldintloss_nomirror_highk.yaml"
# ckpt = "ckpts/dnmpc_1regions_newdata_oldintloss_nomirror_highk/checkpoint_400"

# good l1 one
# config_f = "configs/dnmpc_1regions_newnewdata_l1.yaml"
# ckpt = "ckpts/dnmpc_1regions_newnewdata_l1/checkpoint_3000"

# good l2 one
# config_f = "configs/dnmpc_1regions_newnewdata.yaml"
# ckpt = "ckpts/dnmpc_1regions_newnewdata/checkpoint_5800"

# l1 one only 1 step int
# config_f = "configs/dnmpc_1regions_newnewdata_1stepl1.yaml"
# ckpt = "ckpts/dnmpc_1regions_newnewdata_1stepl1/checkpoint_8400"

# config_f = "configs/dnmpc_1regions_newnewdata_1stepst_l1_newarch_ksint.yaml"
# ckpt = "ckpts/dnmpc_1regions_newnewdata_1stepst_l1_newarch_ksint/checkpoint_5500"

config_f = "configs/dnmpc_1regions_newnewdata_1stepst_l2_newarch_ksint.yaml"
ckpt = "ckpts/dnmpc_1regions_newnewdata_1stepst_l2_newarch_ksint/checkpoint_2900"

# config_f = "configs/dnmpc_1regions_newnewdata_1stepst_l1_newarch_ksint_iq.yaml"
# ckpt = "ckpts/dnmpc_1regions_newnewdata_1stepst_l1_newarch_ksint_iq/checkpoint_5200"

# lower training loss one
# config_f = "configs/dnmpc_1regions_newnewdata_huber0.05.yaml"
# ckpt = "ckpts/dnmpc_1regions_newnewdata_huber0.05/checkpoint_2400"


In [8]:
from importlib import reload
reload(ip)

<module 'irbfn_mpc.irbfn_planner' from '/home/irbfn/src/irbfn_mpc/irbfn_planner.py'>

In [9]:
planner = ip.IRBFNPlanner(config_f, ckpt, track, mirror=True, sv_ind=2)

In [10]:
env.unwrapped.add_render_callback(planner.render_waypoints)
env.unwrapped.add_render_callback(planner.render_goal_state)
env.unwrapped.add_render_callback(planner.render_planner_sol)

In [11]:
import chex
chex.clear_trace_counter()

In [12]:
poses = np.array(
    [
        [
            env.unwrapped.track.raceline.xs[0] - 0.01,
            env.unwrapped.track.raceline.ys[0] - 0.01,
            env.unwrapped.track.raceline.yaws[0],
        ]
    ]
)
obs, info = env.reset(options={"poses": poses})
done = False

step = 0

while not done and step < 1000:
    current_state = obs["agent_0"]
    accl, steerv = planner.plan(current_state)
    action = env.action_space.sample()
    # speed = current_state["linear_vel_x"] + pred_u[0, 0] * 0.1
    # steer = current_state["delta"] + pred_u[0, 5] * 0.1
    action[0] = [steerv, accl]
    # print(f"steerv: {steerv}, accl: {accl}")
    # print("---------------------------------")

    obs, step_reward, done, truncated, info = env.step(action)
    step += 1
env.close()

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")
/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Moviepy - Building video /home/irbfn/scripts/video_irbfn_nmpc_newnewdata/rl-video-episode-0.mp4.
Moviepy - Writing video /home/irbfn/scripts/video_irbfn_nmpc_newnewdata/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/irbfn/scripts/video_irbfn_nmpc_newnewdata/rl-video-episode-0.mp4


In [13]:
import glob
import io
import base64

for video_file in glob.glob("video_irbfn_nmpc_newnewdata/*.mp4"):
    video = io.open(video_file, "rb").read()
    encoded = base64.b64encode(video).decode("ascii")
    display(
        HTML(
            f"""<video width="800" height="auto" controls>
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4" />
            </video>"""
        )
    )